In [1]:
from fnalign.alignment.muse import Embedding, get_emb

In [2]:
en_emb = get_emb("en")

In [3]:
l2_emb = get_emb("es")

In [4]:
from fnalign.loaders import load
from fnalign.models import Alignment

eng_frm = load("bfn", "en")
es_frm = load("spanishfn", "es")
alignment = Alignment("spanishfn", "es")
alignment.load(eng_frm, es_frm)

In [5]:
from collections import defaultdict

import time
start = time.time()
l2_emb.create_index()

vecs = defaultdict(set)
# Including NN in L2 space of english LUs
for name, row in alignment.en_frm.iterrows():
    for lu in row["obj"].lus:
        lu_vec = en_emb.infer_vector(lu.clean_name)
        
        if lu_vec is not None:
            vecs[name].update(l2_emb.get_knn(lu_vec))

# L2 LUs vectors
for name, row in alignment.l2_frm.iterrows():
    for lu in row["obj"].lus:
        if lu.clean_name in l2_emb.word2id:
            vecs[name].add(l2_emb.word2id[lu.clean_name])
    
print("Inference finished --- %s seconds ---" % (time.time() - start))

Loading faiss.


Inference finished --- 280.5107579231262 seconds ---


In [6]:
start = time.time()
scores = []

for frame, other in alignment.pairs():
    if len(frame.lus) == 0:
        scores.append(0)
    else:
        scores.append(len(frm_vecs[frame.name] & frm_vecs[other.name]) / len(frame.lus))

print("Scoring finished --- %s seconds ---" % (time.time() - start))

Scoring finished --- 1.0802419185638428 seconds ---


In [7]:
muse